# **Problema de asignación 3-dimensional axial y 3-dimensional planar.**
### **Introducción:**

* El problema de asignación, consiste en asignar ciertos recuros disponibles a unas tareas, siendo el objetivo relizar esta asignación al menor coste posible. Hay que tener en cuenta, que cada recuerso es destinado a una sola tarea y que cada tarea será ejecutada por un solo recurso. Se trata de un caso particular del problema de transporte, donde la cantidad a repartir desde el origen es 1 y la demanda en cada destino es 1 también.<br><br>

* Como caso de partida, el modelo matemático para el problema de asignación 2-dimensional es el siguiente:<br><br>    

   * ***Variables:***   


      ![](https://drive.google.com/uc?export=view&id=1iRRclx2Oq06Y4IiLbTvrx4Y0Fdh51nwC)

   * ***Función objetivo:*** 

      ![](https://drive.google.com/uc?export=view&id=1iRnQV24gM_yS6hC3jhO9lXtbteTZbila)

  * ***Restricciones:***
      
      ![](https://drive.google.com/uc?export=view&id=1iS9S7dTgsUpq9K3K1ooCXkIixeBRFlkW)
      
      ![](https://drive.google.com/uc?export=view&id=1iToxL1P38we64VoMPhmkECWQmBnsX5wO)
      
      ![](https://drive.google.com/uc?export=view&id=1iUgfD6PqJ-5fTXhmHyxx5JHVe7F172d2)
      
      ![](https://drive.google.com/uc?export=view&id=1iUxACOTwRF0zfBSwuaqDZJHnZR-ZiQ9U)


      ![](https://drive.google.com/uc?export=view&id=17NFk7Ms3Jwyb0YP16xWEhOM92sJBcpzw)


* A continuación se procederá a explicar en qué consiste cada uno y se implementará un código para modelizarlos y resolver un ejemplo, utilizando para ello la herramienta ***OR-tools*** de Google.

### **Problema de asignación 3-dimensional axial:**

* El problema de asignación 3-dimensional axial ***(3AP-axial)*** se trata de una una "*extensión*" del problema de asignación 2-dimensional, donde en este caso se trabaja con 3 conjuntos *I, J y K* **(todos con el mismo número de elementos)**, y obviamente, se busca que se asignen todos los recursos a todas las tareas, de manera que el coste sea mínimo.<br><br>


  ![](https://drive.google.com/uc?export=view&id=1iV76KAlRL6wqDsnz6VuMx9e_VjhN-6Ex)


* Como ejemplo ilustrativo para el 3AP-axial, se plantea un conjunto de transportistas ***(I)***, un conjunto de camiones ***(J)*** y un conjunto de rutas ***(K)***, y se busca que el coste de la asignación entre estos tres conjuntos sea mínimo:<br><br>


  ![](https://drive.google.com/uc?export=view&id=1iW0ozOF3HtmDUt6qFmylt90Sf1fdebRo)

   * ***Variables:***   


   ![](https://drive.google.com/uc?export=view&id=1iXK-o8RgHphRspZ4ArFYbO1t2IAi-Esu)

   * ***Función objetivo:*** 

   ![](https://drive.google.com/uc?export=view&id=1iXxSWdE6pWp6vo6AvSEewv1NpdTjtwzo)
  
  * ***Restricciones:***

      
   ![](https://drive.google.com/uc?export=view&id=1i_M7Ba1RkX6uTOFlSRUpd1KTwwc4wghq)
      
   ![](https://drive.google.com/uc?export=view&id=1ibHVGSsetcoSot82YCgo7iCFwWI4TYcI)
      
   ![](https://drive.google.com/uc?export=view&id=1ibOLkAh6Zqwmme_GvzAjwua7ndLZbzBy)

      
   ![](https://drive.google.com/uc?export=view&id=1idKLVxuddFnTiNuy4vgtnth3UBiVVATp)

      
   ![](https://drive.google.com/uc?export=view&id=1iYvfDT6-frtspzHQdOfeOtqvPGuA9QiW)


In [2]:
!pip install ortools
from ortools.linear_solver import pywraplp
import random 

     |████████████████████████████████| 14.5 MB 92 kB/s 
     |████████████████████████████████| 1.1 MB 49.0 MB/s 
     |████████████████████████████████| 126 kB 65.8 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
  Attempting uninstall: absl-py
    Found existing installation: absl-py 0.12.0
    Uninstalling absl-py-0.12.0:
      Successfully uninstalled absl-py-0.12.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.4.0 requires absl-py<0.13,>=0.9, but you have absl-py 1.0.0 which is incompatible.


In [3]:
dimension = 4 # Cardinal de los conjuntos |I| = |J| = |K| = 5
opciones  = range(dimension)
random.seed(5)
cost = [[[random.randint(1,100) for i in opciones] for j in opciones] for k in opciones]
EPS = 0.001

for i in opciones:
  print("{", end=" ")
  for j in opciones:
    print()
    for k in opciones:
      print(cost[i][j][k], end=" ")
  print("\n}\n")



{ 
80 33 95 46 
89 95 84 68 
4 60 100 32 
84 7 21 15 
}

{ 
48 61 32 49 
70 14 74 32 
2 94 28 53 
36 24 99 50 
}

{ 
21 98 10 18 
80 80 57 17 
17 1 1 27 
100 28 22 22 
}

{ 
38 41 26 70 
87 81 27 24 
89 26 50 39 
3 47 54 22 
}



 * Cabe mencionar que se van a generar tantas matrices de costo como indique la dimensión *(cardinal de los conjuntos)*, y para cada matriz, el número de filas y de columnas será también la dimensión. 

In [ ]:
solver = pywraplp.Solver('3APaxial', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

x = { (i,j,k) : solver.BoolVar('x[%i,%i,%i]' % (i,j,k)) for i in opciones for j in opciones for k in opciones }
            
solver.Minimize( solver.Sum(cost[i][j][k] * x[i,j,k] for i in opciones for j in opciones for k in opciones))

[ solver.Add(solver.Sum(x[i,j,k] for j in opciones for k in opciones) == 1) for i in opciones ]
[ solver.Add(solver.Sum(x[i,j,k] for i in opciones for k in opciones) == 1) for j in opciones ]
[ solver.Add(solver.Sum(x[i,j,k] for i in opciones for j in opciones) == 1) for k in opciones ]
    
sol = solver.Solve()

[print(f'Al conductor {i+1} se le asigna el camión {j+1} para realizar la ruta {k+1}.  Costo = {cost[i][j][k]}') for i in opciones for j in opciones for k in opciones if x[i,j,k].solution_value() > EPS]

print(f'\nEl costo total es {solver.Objective().Value()}. Resuelto en: {solver.WallTime()/1000} segundos')

Al conductor 1 se le asigna el camión 4 para realizar la ruta 5.  Costo = 54
Al conductor 2 se le asigna el camión 2 para realizar la ruta 3.  Costo = 14
Al conductor 3 se le asigna el camión 1 para realizar la ruta 2.  Costo = 136
Al conductor 4 se le asigna el camión 5 para realizar la ruta 1.  Costo = 4
Al conductor 5 se le asigna el camión 3 para realizar la ruta 4.  Costo = 59

El costo total es 267.0. Resuelto en: 0.009 segundos


### **Problema de asignación 3-dimensional planar:**

* El problema de asignación 3-dimensional planar ***(3AP-planar)*** se trata también de una "*extensión*" del problema de asignación 2-dimensional, donde al igual que en caso axial, también se trabaja con 3 conjuntos *I, J y K* **(todos con el mismo número de elementos)**, y obviamente, se busca que se asignen todos los recursos a todas las tareas, de manera que el coste sea mínimo.<br><br>


* Como ejemplo ilustrativo para el 3AP-planar, se plantea buscar la asignación de transportistas, camiones y rutas, de manera que el coste de la asignación total entre estos tres conjuntos sea mínimo. El modo de empleo sería ofrecer a cada transportista las opciones (camión-ruta) con menor coste que se le han asignado, dado que lo que interesa en este problema es minimizar el coste global, no "el personal", de modo que, no pueden haber transportistas que tengan asignado un mismo camión-ruta.<br><br>


  ![](https://drive.google.com/uc?export=view&id=1e_ktI1TK8XB6Y_dZjiLlbjKUcpgW2Hpd)

* Cabe mencionar que para cada transportista se tiene que asignar n camiones y n rutas, existiendo tantas asignaciones como elementos tengan los tres conjuntos, que será n = |I| = |J| = |K|. A continuación se muestra un ejemplo donde el número de elemetos es 4, por lo que habrán 4 conductores (4 matrices), y para cada uno se le asignará 4 pares camión/ruta, de manera que las asignaciones se han realizado minimizando los costes globales: <br><br>


  ![](https://drive.google.com/uc?export=view&id=10CvwKAv10PmbkddrQRbVo9FGcnv4Tqgu)

  ![](https://drive.google.com/uc?export=view&id=1m0X7sulUj_rWl2EXqyjot7345ac5iIlG)

  ![](https://drive.google.com/uc?export=view&id=1AlM7BF7nccOLRjbztjf7Lu__xWo31rjU)

  ![](https://drive.google.com/uc?export=view&id=12PMTe_264O8z0syLlyy6U7u8F1TWYOXm)

* Como se puede observar, las soluciones que se escogen para el problema 3AP-planar corresponden a un **cuadrado latino**, un caso particular (matriz n x n) del **retángulo latino**, que consiste en una matriz n x m donde para cada fila y columna el elemento ij es distinto. Para el caso de las soluciones del problema, la elección que se hace para cada matriz es única en filas y columnas, es decir, no puede haber como solución un mismo elemento en las filas y columnas del conjunto de matrices. El cuadrado latino con las soluciones para el caso de muestra, es el siguiente: <br><br>


  ![](https://drive.google.com/uc?export=view&id=12p_bUqYerczyGGCmN0HW9-aVBUxzGTq7)

  ![](https://drive.google.com/uc?export=view&id=1jXSg6F0YSQ383C1H7d42CeJbU3Zu-O3c)

   * ***Variables:***   



   * ***Función objetivo:*** 

   ![](https://drive.google.com/uc?export=view&id=1iXxSWdE6pWp6vo6AvSEewv1NpdTjtwzo)
  
  * ***Restricciones:***

      
   ![](https://drive.google.com/uc?export=view&id=1_5-n1wloxQlAXvgqW4i2x213F47RQ5QG)
      
   ![](https://drive.google.com/uc?export=view&id=1Ltw3DytoDSP0omRXdK1iTeHfCv2EaLo_)
      
   ![](https://drive.google.com/uc?export=view&id=1_0VU9lsCfXSYGmlEx_jtuSbKk55t50In)
      
   ![](https://drive.google.com/uc?export=view&id=1idKLVxuddFnTiNuy4vgtnth3UBiVVATp)
      
   ![](https://drive.google.com/uc?export=view&id=1iYvfDT6-frtspzHQdOfeOtqvPGuA9QiW)


In [7]:
solver = pywraplp.Solver('3APaxial', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

x = { (i,j,k) : solver.BoolVar('x[%i,%i,%i]' % (i,j,k)) for i in opciones for j in opciones for k in opciones }
            
solver.Minimize( solver.Sum(cost[i][j][k] * x[i,j,k] for i in opciones for j in opciones for k in opciones))

[ solver.Add(solver.Sum(x[i,j,k] for i in opciones) == 1) for j in opciones for k in opciones ]
[ solver.Add(solver.Sum(x[i,j,k] for j in opciones) == 1) for i in opciones for k in opciones ]
[ solver.Add(solver.Sum(x[i,j,k] for k in opciones) == 1) for i in opciones for j in opciones ]
    
sol = solver.Solve()


for i in opciones:
    for j in opciones:
        for k in opciones:
            if x[i,j,k].solution_value() > EPS :
                print(f'Al transportista {i+1} se le ha asignado el camión {j+1} para la ruta {k+1}. Coste = {cost[i][j][k]}')
    print("\n")

print(f'\nEl costo total es {solver.Objective().Value()}. Resuelto en: {solver.WallTime()/1000} segundos')

Al transportista 1 se le ha asignado el camión 1 para la ruta 2. Coste = 33
Al transportista 1 se le ha asignado el camión 2 para la ruta 1. Coste = 89
Al transportista 1 se le ha asignado el camión 3 para la ruta 4. Coste = 32
Al transportista 1 se le ha asignado el camión 4 para la ruta 3. Coste = 21


Al transportista 2 se le ha asignado el camión 1 para la ruta 3. Coste = 32
Al transportista 2 se le ha asignado el camión 2 para la ruta 2. Coste = 14
Al transportista 2 se le ha asignado el camión 3 para la ruta 1. Coste = 2
Al transportista 2 se le ha asignado el camión 4 para la ruta 4. Coste = 50


Al transportista 3 se le ha asignado el camión 1 para la ruta 1. Coste = 21
Al transportista 3 se le ha asignado el camión 2 para la ruta 4. Coste = 17
Al transportista 3 se le ha asignado el camión 3 para la ruta 3. Coste = 1
Al transportista 3 se le ha asignado el camión 4 para la ruta 2. Coste = 28


Al transportista 4 se le ha asignado el camión 1 para la ruta 4. Coste = 70
Al trans